In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
import pandas as pd
import numpy as np

import edhek_risk_kit as erk

ind = erk.get_hfi_returns()["2000":]
er = erk.annualize_rets(ind, 12)
cov = ind.cov()

In [23]:
def display_as_pct(n):
    if isinstance(n, np.float64) or isinstance(n, np.float32) or isinstance(n, np.float16):
        return display_as_pct(n.item())    
    elif isinstance(n, float):
        return round(n * 100, 2)


In [25]:
display_as_pct(erk.var_gaussian(ind["Distressed Securities"], level=1, modified=False))

3.14

In [26]:
display_as_pct(erk.var_gaussian(ind["Distressed Securities"], level=1, modified=True))

4.97

In [27]:
display_as_pct(erk.var_historic(ind["Distressed Securities"], level=1))

4.26

In [70]:
ind = erk.get_ind_returns()
start_year, end_year = ("2013", "2017")
er = erk.annualize_rets(ind[start_year:end_year], 12)
cov = ind[start_year:end_year].cov()

In [71]:
industries = ["Books", "Steel", "Oil", "Mines"]

In [72]:
er[industries]

Books    0.095312
Steel    0.100716
Oil      0.025275
Mines    0.005136
dtype: float64

In [73]:
cov.loc[industries, industries]

,Books,Steel,Oil,Mines
Books,0.002720,0.001940,0.001637,0.001539
Steel,0.001940,0.004391,0.002122,0.003391
Oil,0.001637,0.002122,0.002697,0.002179
Mines,0.001539,0.003391,0.002179,0.005232


In [38]:
n = er[industries].shape[0]
w_ew = np.repeat(1/n, n)
w_ew = pd.Series(w_ew, index=er[industries].index)
w_ew * 100

Books    25.0
Steel    25.0
Oil      25.0
Mines    25.0
dtype: float64

In [47]:
riskfree_rate = 0.1
w_msr = erk.msr(riskfree_rate, er[industries], cov.loc[industries, industries])
w_msr = pd.Series(w_msr, index=er[industries].index) * 100
print(round(w_msr, 2))

Books      0.0
Steel    100.0
Oil        0.0
Mines      0.0
dtype: float64


In [48]:
w_gmv = erk.gmv(cov.loc[industries, industries])
w_gmv = pd.Series(w_gmv, index=er[industries].index)
print(round(w_gmv * 100, 2).sort_values())

Steel     0.00
Mines     8.89
Oil      43.41
Books    47.70
dtype: float64


In [74]:
w_msr = erk.msr(riskfree_rate, er[industries], cov.loc[industries, industries])
w_gmv = erk.gmv(cov.loc[industries, industries])

In [75]:
ind_2018 = ind["2018":"2018"][industries]
ind_2018

,Books,Steel,Oil,Mines
2018-01,0.0449,0.0361,0.0286,0.0357
2018-02,-0.0325,-0.0199,-0.1102,-0.0335
2018-03,0.0031,-0.0586,0.0308,-0.0015
2018-04,-0.0062,0.0366,0.1030,-0.0339
2018-05,-0.0061,0.0562,0.0343,0.0308
2018-06,0.0218,-0.0514,0.0052,-0.0178
2018-07,-0.0417,0.0424,0.0104,-0.0224
2018-08,-0.0447,-0.0323,-0.0295,-0.1075
2018-09,-0.0221,-0.0067,0.0356,-0.0173
2018-10,-0.0666,-0.1161,-0.1206,-0.1032


In [76]:
er_2018 = erk.annualize_rets(ind_2018, 12)
cov_2018 = ind_2018.cov()
cov_2018

,Books,Steel,Oil,Mines
Books,0.002799,0.002098,0.002558,0.001629
Steel,0.002098,0.004027,0.003473,0.001843
Oil,0.002558,0.003473,0.005296,0.001961
Mines,0.001629,0.001843,0.001961,0.002059


In [80]:
vol_msr = erk.portfolio_vol(w_msr, cov_2018)
vol_msr_annual = vol_msr * np.sqrt(12)
print(round(vol_msr_annual * 100, 2))

21.98


In [81]:
vol_gmv = erk.portfolio_vol(w_gmv, cov_2018)
vol_gmv_annual = vol_gmv * np.sqrt(12)
print(round(vol_gmv_annual * 100, 2))

18.97
